# Processing WDS Catalog

The [Washington Double Star Catalog (WDS)](http://www.usno.navy.mil/USNO/astrometry/optical-IR-prod/wds/WDS) is the world's principal database of astrometric double and multiple star information.

The objective is to process WDS data and obtain the coordinates from double stars following some criteria:

 - Last observation must be at least 1975
 - Both stars magnitudes must be 19 or less.
 - Separation must be between 2 and 180.
 - Proper motion must be greater or equal to 60.

If any double star matches the criteria -> 2000 arcsecond coordinates will be stored.

[Catalog format](http://ad.usno.navy.mil/wds/Webtextfiles/wdsweb_format.txt) is set by the WDS.

In [1]:
import urllib.request as req
import shutil as shu
import os
import math as mth

WDS catalog is stored online.

In [2]:
url = 'http://ad.usno.navy.mil/wds/Webtextfiles/wdsweb_summ.txt'

Once a star matches it will be stored but there are several modes to store:

 - **a**: all the stars are stored in the same file.
 - **t**: the stars are splited by their theta, position angle, value.
 - **r**: the split is done according to rho, separation, value.

In [3]:
mode = 'a'
split = 5

Criteria will be checked for every line of data in the WDS catalog.

In [4]:
def match(data):
    try:
        return int(data[28:32]) > 1975 and 2 < float(data[52:57]) < 180 and \
               mth.sqrt(int(data[80:84]) ** 2 + int(data[84:88]) ** 2) >= 60 and \
               mth.sqrt(int(data[89:93]) ** 2 + int(data[93:97]) ** 2) >= 60 and \
               float(data[58:63]) < 19 and float(data[64:69]) < 19
    except ValueError:
        return False

If the star matched the criteria it will be assigned to a directory, according to mode and split.

In [5]:
def directory(theta, rho, split, mode):
    theta = round(float(theta))
    rho = round(float(rho))

    group = (theta // split) * split

    return { 'a': 'WDS/All',
             't': 'WDS/Theta_{0}_{1}'.format(group, group + split),
             'r': 'WDS/Rho_{0}_{1}'.format(group, group + split) }[mode]

Star 2000 arcsen coordinates must be formated.

In [6]:
def coord_format(coord):
    list = []

    list.append(coord[:2])
    list.append(coord[2:4])
    list.append(coord[4:9])

    list.append(coord[9:12])
    list.append(coord[12:14])
    list.append(coord[14:])

    return ' '.join(list)

The directory must be cleared before the data is processed.

In [7]:
print("Cleaning WDS directory...")
shu.rmtree("WDS", ignore_errors=True)

Cleaning WDS directory...


Now that all the process have been prepared, it's time to download WDS data and process it line by line.

In [8]:
print("Trying to download data from {0}...".format(url))
try:
    data = req.urlopen(url).readlines()
    print("Processing data...")
    for line in data:
        if match(line):
            dir = directory(line[38:41], line[46:51], split, mode)
            os.makedirs(dir, exist_ok=True)
            
            out = open('{0}/wds_coord.txt'.format(dir), 'a+')
            out.write(coord_format(line[112:].decode()))
            out.close()
    print("Done")
except req.HTTPError:
    print("URL not found")

Trying to download data from http://ad.usno.navy.mil/wds/Webtextfiles/wdsweb_summ.txt...
Processing data...
Done
